<a href="https://colab.research.google.com/github/ChagataiDuru/CS304-IntroAI-CollabNotebooks/blob/main/CS304_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2

In [4]:
# Importing Necessary Libraries and File for Part 2 soft-margin SVM to handwritten digits
from os.path import exists
import itertools as itertools

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

train_data = "/content/drive/MyDrive/Colab Notebooks/CS304/HW2/features_train.txt"
test_data = "/content/drive/MyDrive/Colab Notebooks/CS304/HW2/features_test.txt"

# - Supress all warnings (Optional)
import warnings
warnings.simplefilter(action='ignore',
                      category=FutureWarning)
if exists(train_data) and exists(test_data):
    print(f"\nBoth {train_data} and {test_data} exists.")
else:
    print("Please set directory to read the files")

from sklearn.svm import SVC


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Both /content/drive/MyDrive/Colab Notebooks/CS304/HW2/features_train.txt and /content/drive/MyDrive/Colab Notebooks/CS304/HW2/features_test.txt exists.


## Part 1 #
Consider the nonlinear error surface $E(u, v)=\left(u e^{v}-2 v e^{-u}\right)^{2}$. We start at the point $$(u, v)=(1,1)$$ and minimize this error using gradient descent in the $u v$ space. Use $$\eta=0.1$$ (learning rate, not step size).
## Question 1 ##
What is the partial derivative of $E(u, v)$ with respect to $u$, i.e., $\frac{\partial E}{\partial u}$ ?

**Answer:**

**Error Surface:**

$$E(u, v) = \left(u e^{v}-2 v e^{-u}\right)^{2}$$

**Partial Derivative with Respect to u:**

$$\frac{\partial E}{\partial u} = 2\left(e^{v}+2 v e^{-u}\right)\left(u e^{v}-2 v e^{-u}\right)$$

##Question 2##
How many iterations (among given choices) does it take for the error $E(u, v)$ to fall below $10^{-14}$ for the first time? In your programs, make sure to use double precision to get the needed accuracy.

(a) 1

(b) 3

(c) 5

(d) 10

(e) 17

**Answer: d**

##Question 2##
After running enough iterations such that the error has just dropped below $10^{-14}$, what are the closest values (in Euclidean distance) among the following choices to the final $(u, v)$ you got in problem 2 ?

(a) $(1.000,1.000)$

(b) $(0.713,0.045)$

(c) $(0.016,0.112)$

(d) $(-0.083,0.029)$

(e) $(0.045,0.024)$


**Answer: e**

### Below the code ###

In [5]:
def error_surface(u, v):
    return (u * np.exp(v) - 2 * v * np.exp(-u))**2

def dE_du(u, v):
    return 2 * (np.exp(v) + 2 * v * np.exp(-u)) * (u * np.exp(v) - 2 * v * np.exp(-u))

def dE_dv(u, v):
    return 2 * u * np.exp(v) * (u * np.exp(v) - 2 * v * np.exp(-u))

def gradient_descent(E, dE_du, dE_dv, u_0, v_0, learning_rate = 0.1,
                     stopping_threshold = 1e-14):
    u = u_0
    v = v_0
    iterations = 0

    while E(u, v) >= stopping_threshold:
        u_new = u - learning_rate * dE_du(u, v)
        v_new = v - learning_rate * dE_dv(u, v)
        u = u_new
        v = v_new
        iterations += 1

    return u, v, iterations

u_0 = 1
v_0 = 1
eta = 0.1
threshold = 10**(-14)

final_u, final_v, iterations = gradient_descent(error_surface, dE_du, dE_dv, u_0, v_0, eta, threshold)

print("Final values (u, v): ", final_u, final_v)
print("Iterations to converge: ", iterations)


Final values (u, v):  -0.3153488929785157 -0.10369822395113008
Iterations to converge:  49
